### Part I – Code

In [ ]:
# Dataset Selection and Preprocessin
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

# Loadingn the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print("Part I: Dataset Selection")
print(f"Number of samples: {X.shape[0]}")
print(f"Number of features: {X.shape[1]}")
print(f"Number of classes: {len(np.unique(y))}")


Part I: Dataset Selection
Number of samples: 150
Number of features: 4
Number of classes: 3


### Part I – Conclusions
- The Iris dataset from the UCI repository was chosen for its simplicity and suitability for classification tasks. With 150 samples, 4 features (sepal length, sepal width, petal length, petal width), and 3 classes (Setosa, Versicolor, Virginica), it provides a solid foundation for testing the neural network and ensemble methods implemented in this homework.

### Part II – Code

In [ ]:
# AdaBoost with MLP as base classifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

# Define MLP classifier that supports sample weights
class KerasMLPClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, hidden_layer_size=10, max_iter=1000, random_state=42):
        self.hidden_layer_size = hidden_layer_size
        self.max_iter = max_iter
        self.random_state = random_state
        self.model = None

    def fit(self, X, y, sample_weight=None):
        np.random.seed(self.random_state)
        tf.random.set_seed(self.random_state)
        self.classes_ = np.unique(y)
        num_classes = len(self.classes_)
        self.model = Sequential([
            Dense(self.hidden_layer_size, activation='relu', input_shape=(X.shape[1],)),
            Dense(num_classes, activation='softmax')
        ])
        self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        self.model.fit(X, y, epochs=self.max_iter, verbose=0, sample_weight=sample_weight)
        return self

    def predict(self, X):
        probabilities = self.model.predict(X, verbose=0)
        return np.argmax(probabilities, axis=1)

# Train AdaBoost ensemble with MLP base classifier
base_classifier = KerasMLPClassifier()
ada_boost = AdaBoostClassifier(estimator=base_classifier, n_estimators=50, random_state=42)
ada_boost.fit(X_train, y_train)
y_pred_ada = ada_boost.predict(X_test)

print("\nPart II: AdaBoost with MLP")
print("Accuracy:", accuracy_score(y_test, y_pred_ada))
print("Classification Report:")
print(classification_report(y_test, y_pred_ada))

/Users/aykutss/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Part II: AdaBoost with MLP
Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



### Part II – Conclusions
- The AdaBoost model, utilizing a custom MLP with one hidden layer (10 neurons) as the base classifier, demonstrated perfect classification performance on the Iris dataset. The ensemble approach enhanced the MLP’s ability to handle complex decision boundaries, resulting in 100% accuracy, with precision, recall, and F1-scores of 1.00 across all classes.

### Part III – Code

In [ ]:
# Custom Random Forest with Perceptron-based Nodes
from sklearn.linear_model import LogisticRegression
from scipy.stats import mode

class Node:
    def __init__(self, perceptron=None, feature_indices=None, left=None, right=None, prediction=None):
        self.perceptron = perceptron
        self.feature_indices = feature_indices
        self.left = left
        self.right = right
        self.prediction = prediction

class CustomDecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2, num_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.num_features = num_features
        self.root = None

    def fit(self, X, y):
        self.num_features = X.shape[1] if self.num_features is None else min(self.num_features, X.shape[1])
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        if depth >= self.max_depth or num_samples < self.min_samples_split or len(np.unique(y)) == 1:
            return Node(prediction=np.bincount(y).argmax())
        feature_indices = np.random.choice(num_features, self.num_features, replace=False)
        best_gini, best_split = float('inf'), None
        unique_classes = np.unique(y)
        for class_ in unique_classes:
            A = [class_]
            B = [c for c in unique_classes if c != class_]
            y_binary = np.where(np.isin(y, A), 0, 1)
            perceptron = LogisticRegression(max_iter=1000)
            perceptron.fit(X[:, feature_indices], y_binary)
            predictions = perceptron.predict(X[:, feature_indices])
            left_idx, right_idx = np.where(predictions == 0)[0], np.where(predictions == 1)[0]
            if len(left_idx) == 0 or len(right_idx) == 0:
                continue
            counts_left = np.bincount(y[left_idx], minlength=len(unique_classes))
            counts_right = np.bincount(y[right_idx], minlength=len(unique_classes))
            gini_left = 1 - sum((counts_left / max(1, len(left_idx)))**2)
            gini_right = 1 - sum((counts_right / max(1, len(right_idx)))**2)
            gini = (len(left_idx) / num_samples) * gini_left + (len(right_idx) / num_samples) * gini_right
            if gini < best_gini:
                best_gini, best_split = gini, (perceptron, feature_indices)
        if best_split is None:
            return Node(prediction=np.bincount(y).argmax())
        perceptron, feature_indices = best_split
        predictions = perceptron.predict(X[:, feature_indices])
        left_tree = self._grow_tree(X[predictions == 0], y[predictions == 0], depth + 1)
        right_tree = self._grow_tree(X[predictions == 1], y[predictions == 1], depth + 1)
        return Node(perceptron=perceptron, feature_indices=feature_indices, left=left_tree, right=right_tree)

    def predict(self, X):
        return np.array([self._predict_single(x, self.root) for x in X])

    def _predict_single(self, x, node):
        if node.prediction is not None:
            return node.prediction
        pred = node.perceptron.predict(x[node.feature_indices].reshape(1, -1))[0]
        return self._predict_single(x, node.left if pred == 0 else node.right)

class CustomRandomForest:
    def __init__(self, n_trees=10, max_depth=5, min_samples_split=2, num_features=2):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.num_features = num_features
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.n_trees):
            idx = np.random.choice(len(X), len(X), replace=True)
            tree = CustomDecisionTree(self.max_depth, self.min_samples_split, self.num_features)
            tree.fit(X[idx], y[idx])
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        return mode(predictions, axis=0)[0].flatten()

forest = CustomRandomForest()
forest.fit(X_train, y_train)
y_pred_rf = forest.predict(X_test)
print("\nPart III: Custom Random Forest with Perceptron Decisions")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:")
print(classification_report(y_test, y_pred_rf))


Part III: Custom Random Forest with Perceptron Decisions
Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



### Part III – Conclusions
- The custom random forest model, using perceptron-like logistic regressors as decision nodes, achieved perfect classification performance on the Iris dataset. By leveraging oblique decision boundaries instead of standard threshold-based splits, the model was able to effectively separate the classes. This demonstrates the strength of using trainable models at the node level within ensemble methods, especially for small and clean datasets.